In [35]:
import pandas as pd
import csv
from google.colab import drive
import numpy as np
from numpy import float64, int32 

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
metadata= pd.read_csv('/content/drive/MyDrive/hybridrecommendersystem/movies_metadata.csv')
ratings= pd.read_csv('/content/drive/MyDrive/hybridrecommendersystem/ratings.csv', dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
#   metadata=pd.read_csv('/content/drive/MyDrive/hybridrecommendationsystem/movies_metadata.csv/movies_metadata.csv')
#   ratings=pd.read_csv('/content/drive/MyDrive/hybridrecommendationsystem/ratings.csv/ratings.csv')

In [38]:

metadata.sort_values(by= ['vote_count'], ascending=False,inplace=True)
metadata.reset_index(inplace=True)
metadata

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,15480,False,NaN,160000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",http://inceptionmovie.warnerbros.com/,27205,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.1081,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2010-07-14,8.255328e+08,148.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your mind is the scene of the crime.,Inception,False,8.1,14075.0
1,12481,False,"{'id': 263, 'name': 'The Dark Knight Collectio...",185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://thedarkknight.warnerbros.com/dvdsite/,155,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.167,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,"[{'name': 'DC Comics', 'id': 429}, {'name': 'L...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2008-07-16,1.004558e+09,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269.0
2,14551,False,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.071,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2.787965e+09,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.2,12114.0
3,17818,False,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",http://marvel.com/avengers_movie/,24428,tt0848228,en,The Avengers,When an unexpected enemy emerges and threatens...,89.8876,/cezWGskPY5x7GaglTTRN4Fugfb8.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-04-25,1.519558e+09,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Some assembly required.,The Avengers,False,7.4,12000.0
4,26564,False,"{'id': 448150, 'name': 'Deadpool Collection', ...",58000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.foxmovies.com/movies/deadpool,293660,tt1431045,en,Deadpool,Deadpool tells the origin story of former Spec...,187.86,/inVq3FRqcYIRl2la8iZikYYxFNR.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2016-02-09,7.831130e+08,108.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Witness the beginning of a happy ending,Deadpool,False,7.4,11444.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45462,29502,False,"{'id': 122661, 'name': 'Mardock Scramble Colle...",0,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",http://m-scramble.jp/exhaust/,122662,tt2423504,ja,マルドゥック・スクランブル 排気,Third film of the Mardock Scramble series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45463,29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,False,7.0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45464,35586,False,NaN,0,"[{'id': 10770

In [39]:
metadata = metadata.replace(np.nan, ' ', regex=True)
for index, row in metadata.iterrows():
    if row['title']==' ' or row['vote_count']==0:
        metadata.drop(index, inplace=True)

In [40]:
metadata['popularity'] = metadata['popularity'].astype(float64)
thres = metadata.popularity.quantile(0.9)
metadata = metadata.query('popularity >= @thres')
# metadata

In [41]:
metadata_1=metadata.drop(columns=['homepage','original_title','poster_path','tagline','video','status','runtime','budget','homepage','vote_average'	,'vote_count','imdb_id','popularity', 'production_companies','production_countries','spoken_languages','release_date','revenue','video','belongs_to_collection'])

In [43]:
l1 = [' ']*len(metadata_1['id'])
metadata_1['genre'] = l1
metadata_1.reset_index(inplace= True, drop=False)

In [44]:
row=0
for i in metadata_1['genres']:
    if(i.find("War")!=-1):
        # metadata_1['War'][row]=1
        metadata_1['genre'][row]+='War '
    if(i.find("Action")!=-1):
        # metadata_1['Action'][row]=1
        metadata_1['genre'][row]+='Action '
    if(i.find("Adventure")!=-1):
        # metadata_1['Adventure'][row]=1
        metadata_1['genre'][row]+='Adventure '
    if(i.find("Romance")!=-1):
        # metadata_1['Romance'][row]=1
        metadata_1['genre'][row]+='Romance '
    if(i.find("Drama")!=-1):
        # metadata_1['Drama'][row]=1
        metadata_1['genre'][row]+='Drama '
    if(i.find("Horror")!=-1):
        # metadata_1['Horror'][row]=1
        metadata_1['genre'][row]+='Horror '
    if(i.find("Fantasy")!=-1):
        # metadata_1['Fantasy'][row]=1
        metadata_1['genre'][row]+='Fantasy '
    if(i.find("Thriller")!=-1):
        # metadata_1['Thriller'][row]=1
        metadata_1['genre'][row]+='Thriller '
    if(i.find("Family")!=-1):
        # metadata_1['Family'][row]=1
        metadata_1['genre'][row]+='Family '
    if(i.find("Mystery")!=-1):
        # metadata_1['Mystery'][row]=1
        metadata_1['genre'][row]+='Mystery '
    if(i.find("Science Fiction")!=-1):
        # metadata_1['Science Fiction'][row]=1
        metadata_1['genre'][row]+='Science Fiction '
    if(i.find("Documentary")!=-1):
        # metadata_1['Documentary'][row]=1
        metadata_1['genre'][row]+='Documentary '
    if(metadata_1['adult'][row]=='True'):
        metadata_1['genre'][row]+='Adult'
    row+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_lau

In [45]:
metadata_1['genre']+=metadata_1['original_language']
metadata_1['overview']+=metadata_1['genre']

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vector = TfidfVectorizer(stop_words='english')
metadata_1['overview'] = metadata_1['overview'].fillna('')
tfidf_matrix = tfidf_vector.fit_transform(metadata_1['overview'])

from sklearn.metrics.pairwise import linear_kernel
sim_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(metadata_1.index, index=metadata_1['title']).drop_duplicates()

In [48]:
movies_df = metadata.drop(columns = ['adult', 'genres', 'original_language', 'overview','homepage','original_title','poster_path','tagline','video','status','runtime','budget','homepage','imdb_id', 'production_companies','production_countries','spoken_languages','release_date','revenue','video','belongs_to_collection'])
movies_df.drop(['index', 'vote_average', 'popularity','vote_count'], axis=1, inplace=True)
movies_df.rename(columns = {"id":"movieId"}, inplace = True)

In [49]:
movies_df1 = ratings.drop(['timestamp'], axis=1)

In [50]:
movies_df['movieId'] = movies_df['movieId'].astype(int32)
movies_df = movies_df.merge(movies_df1, left_on = 'movieId', right_on = 'movieId', how = 'left')
movies_df = movies_df.replace(np.nan, ' ', regex=True)
movies_df

,movieId,title,userId,rating
0,27205,Inception,41190,3
1,27205,Inception,45811,4
2,27205,Inception,201372,3
3,27205,Inception,225396,4.5
4,27205,Inception,243331,4
...,...,...,...,...
4738652,26550,Little Lips,248978,3
4738653,26550,Little Lips,258205,3
4738654,26550,Little Lips,267535,3.5
4738655,26550,Little Lips,267772,2


In [51]:
combine_movie_rating = movies_df.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (combine_movie_rating.
    groupby(by = ['title'])['rating'].
    count().
    reset_index().
    rename(columns = {'rating': 'totalRatingCount'})
    [['title', 'totalRatingCount']]
    )
movies_df = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
movies_df.head(10)

,movieId,title,userId,rating,totalRatingCount
0,27205,Inception,41190,3,5
1,27205,Inception,45811,4,5
2,27205,Inception,201372,3,5
3,27205,Inception,225396,4.5,5
4,27205,Inception,243331,4,5
5,155,The Dark Knight,12,1,1319
6,155,The Dark Knight,346,4,1319
7,155,The Dark Knight,792,2,1319
8,155,The Dark Knight,845,5,1319
9,155,The Dark Knight,1460,3,1319


In [52]:
selected_columns = metadata[["vote_average","id"]]
new_df = selected_columns.copy()
new_df.rename(columns = {"id":"movieId"}, inplace = True)
new_df['movieId'] = new_df['movieId'].astype(int32)
movies_df = movies_df.merge(new_df, left_on ='movieId', right_on = 'movieId', how='left')
movies_df

,movieId,title,userId,rating,totalRatingCount,vote_average
0,27205,Inception,41190,3,5,8.1
1,27205,Inception,45811,4,5,8.1
2,27205,Inception,201372,3,5,8.1
3,27205,Inception,225396,4.5,5,8.1
4,27205,Inception,243331,4,5,8.1
...,...,...,...,...,...,...
4738718,26550,Little Lips,248978,3,94,5.8
4738719,26550,Little Lips,258205,3,94,5.8
4738720,26550,Little Lips,267535,3.5,94,5.8
4738721,26550,Little Lips,267772,2,94,5.8


In [53]:
movies_df['userId'].replace(' ',np.nan, regex=True, inplace = True)
movies_df = movies_df[movies_df['userId'].notna()]
movies_df['rating'] = movies_df['rating'].astype(int32)
movies_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,movieId,title,userId,rating,totalRatingCount,vote_average
0,27205,Inception,41190.0,3,5,8.1
1,27205,Inception,45811.0,4,5,8.1
2,27205,Inception,201372.0,3,5,8.1
3,27205,Inception,225396.0,4,5,8.1
4,27205,Inception,243331.0,4,5,8.1
...,...,...,...,...,...,...
4738718,26550,Little Lips,248978.0,3,94,5.8
4738719,26550,Little Lips,258205.0,3,94,5.8
4738720,26550,Little Lips,267535.0,3,94,5.8
4738721,26550,Little Lips,267772.0,2,94,5.8


In [54]:
movie_features_df=movies_df.pivot_table(index='title',columns='userId',values='rating').fillna(0)
# movie_features_df.describe()

In [55]:
movie_features_df

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,40.0,41.0,...,270854.0,270855.0,270856.0,270857.0,270858.0,270859.0,270860.0,270861.0,270863.0,270864.0,270865.0,270866.0,270867.0,270868.0,270869.0,270870.0,270871.0,270872.0,270874.0,270875.0,270877.0,270878.0,270879.0,270880.0,270881.0,270882.0,270883.0,270884.0,270885.0,270886.0,270887.0,270888.0,270889.0,270890.0,270891.0,270892.0,270893.0,270894.0,270895.0,270896.0
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000 BC",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13 Sins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zathura: A Space Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zodiac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
eXistenZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
from scipy.sparse import csr_matrix
movie_features_df_matrix = csr_matrix(movie_features_df.values)
from sklearn.neighbors import NearestNeighbors
model_knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                 radius=1.0)

In [58]:
movie_features_df.shape

(1270, 246737)

In [68]:
def hybrid_recom(movie_title, sim_scores=sim_matrix, indices=indices):
    try:
        idx = indices[movie_title]
        sim_scores = list(enumerate(sim_matrix[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:501]
        movie_indices = [i[0] for i in sim_scores]
        l1 = metadata_1['title'].iloc[movie_indices].tolist()
        df1 = movie_features_df.copy(deep=True)
        df1['new_col'] = range(0, len(df1))
        query_index_name=df1[df1.index == movie_title]['new_col']
        query_index=query_index_name[0]
        distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 500)
        l2=[] 
        for i in range(1, len(distances.flatten())):
            l2.append(movie_features_df.index[indices.flatten()[i]])
        d = dict()
        for i  in range(0,len(l1)):
            flag=1
            for j in range(0,len(l2)):
                if l1[i]==l2[j]:
                    d[l1[i]]=i+2*j;
                    flag=0
            if flag:
                d[l1[i]]=3*i;
        print("The Recommended Movies for {} are:".format(movie_title))
        n = 0
        for i in sorted(d.items(), key=lambda x: x[1]):
            print(i[0])
            if n==9:
                break
            n+=1
    except:
        print("Sorry, couldn't find anything for the movie you requested")

In [64]:
movie_title = str(input())

Inception


In [69]:
hybrid_recom(movie_title)

The Recommended Movies for Inception are:
Altitude
Mission: Impossible - Rogue Nation
House
Renaissance
Flatliners
Central Intelligence
The Fifth Element
Morgan
Pandorum
Pitch Perfect 2
Where Eagles Dare
